### 1. Import thư viện

In [37]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, root_mean_squared_error
import warnings
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules, apriori

warnings.filterwarnings('ignore')

### 2. Tiền xử lý dữ liệu

In [ ]:
# Đọc dữ liệu
df = pd.read_csv("datasets/du_lieu_diem_thi_2017_2024.csv")

# Đổi tên cột cho dễ làm việc
df.columns = ['nam', 'ma_tinh', 'sbd', 'toan', 'van', 'ly', 'hoa', 'sinh',
              'su', 'dia', 'gdcd', 'nn', 'ma_nn', 'tong_diem',
              'khoi_a', 'khoi_a1', 'khoi_b', 'khoi_c', 'khoi_d']

# Chuyển các cột điểm về kiểu số
cols_diem = ['toan', 'van', 'ly', 'hoa', 'sinh', 'su', 'dia', 'gdcd', 'nn',
             'tong_diem', 'khoi_a', 'khoi_a1', 'khoi_b', 'khoi_c', 'khoi_d']
df[cols_diem] = df[cols_diem].apply(pd.to_numeric, errors = 'coerce')

# Tạo cột tỉnh từ mã tỉnh
ma_tinh = {
    '01': 'Hà Nội', '02': 'TP. Hồ Chí Minh', '03': 'Hải Phòng', '04': 'Đà Nẵng',
    '05': 'Hà Giang', '06': 'Cao Bằng', '07': 'Lai Châu', '08': 'Lào Cai',
    '09': 'Tuyên Quang', '10': 'Lạng Sơn', '11': 'Bắc Kạn', '12': 'Thái Nguyên',
    '13': 'Yên Bái', '14': 'Sơn La', '15': 'Phú Thọ', '16': 'Vĩnh Phúc',
    '17': 'Quảng Ninh', '18': 'Bắc Giang', '19': 'Bắc Ninh', '21': 'Hải Dương',
    '22': 'Hưng Yên', '23': 'Hòa Bình', '24': 'Hà Nam', '25': 'Nam Định',
    '26': 'Thái Bình', '27': 'Ninh Bình', '28': 'Thanh Hóa', '29': 'Nghệ An',
    '30': 'Hà Tĩnh', '31': 'Quảng Bình', '32': 'Quảng Trị', '33': 'Thừa Thiên-Huế',
    '34': 'Quảng Nam', '35': 'Quảng Ngãi', '36': 'Kon Tum', '37': 'Bình Định',
    '38': 'Gia Lai', '39': 'Phú Yên', '40': 'Đắk Lắk', '41': 'Khánh Hòa',
    '42': 'Lâm Đồng', '43': 'Bình Phước', '44': 'Bình Dương', '45': 'Ninh Thuận',
    '46': 'Tây Ninh', '47': 'Bình Thuận', '48': 'Đồng Nai', '49': 'Long An',
    '50': 'Đồng Tháp', '51': 'An Giang', '52': 'Bà Rịa-Vũng Tàu', '53': 'Tiền Giang',
    '54': 'Kiên Giang', '55': 'Cần Thơ', '56': 'Bến Tre', '57': 'Vĩnh Long',
    '58': 'Trà Vinh', '59': 'Sóc Trăng', '60': 'Bạc Liêu', '61': 'Cà Mau',
    '62': 'Điện Biên', '63': 'Đắk Nông', '64': 'Hậu Giang'
}

df['ma_tinh_str'] = df['ma_tinh'].astype(str).str.zfill(2)
df['tinh'] = df['ma_tinh_str'].map(ma_tinh)

print("Số dòng ban đầu:", len(df))

# Loại bỏ dòng không có điểm Toán, Văn, Ngoại ngữ (3 Môn thi bắt buộc nên nếu không có thì khả năng cao là không hợp lệ)
df = df[df[['toan', 'van', 'nn']].notna().all(axis = 1)]
print("Số dòng sau khi loại bỏ dòng không có điểm Toán, Văn, Ngoại ngữ:", len(df))

# Lọc học sinh không có tổng điểm
df = df[df['tong_diem'].notna()]
print(f"Số dòng sau khi lọc học sinh không có tổng điểm: {len(df)}")

# Kiểm tra và loại bỏ dòng trùng lặp
df = df.drop_duplicates()
print(f"Số dòng sau khi loại bỏ dòng trùng lặp: {len(df)}")

# Shuffle dữ liệu
df = df.sample(frac = 1, random_state = 42).reset_index(drop = True)

df.head(10)
print(df.head(10).to_markdown())

### 3. Thí nghiệm 3

Các bước phân tích tập phổ biến:
1. Khởi tạo DataFrame Boolean cho các môn học, đánh dấu học sinh giỏi cho từng môn (điểm >= 7.0)
2. Sử dụng thuật toán Apriori để tìm các tập phổ biến
3. Tìm luật kết hợp từ các tập phổ biến

In [72]:
import pandas as pd

subjects = ['toan','ly','hoa','van','su','dia','sinh','gdcd','nn']

# 1) df_pass: môn nào >= 7 là True, NaN → False
df_pass = df[subjects].ge(7).fillna(False).astype(bool)
df_pass.columns = [f'{mon}_gioi' for mon in df_pass.columns]

# 3) Ghép lại vào df gốc
df_bool = pd.concat([df, df_pass], axis=1)

df_bool.head(10)

,nam,ma_tinh,sbd,toan,van,ly,hoa,sinh,su,dia,...,tinh,toan_gioi,ly_gioi,hoa_gioi,van_gioi,su_gioi,dia_gioi,sinh_gioi,gdcd_gioi,nn_gioi
0,17,2,2464,8.2,4.50,8.00,5.25,5.0,NaN,NaN,...,TP. Hồ Chí Minh,True,True,False,False,False,False,False,False,True
1,23,48,9389,8.4,5.00,7.50,8.00,4.5,NaN,NaN,...,Đồng Nai,True,True,True,False,False,False,False,False,False
2,22,49,8861,4.8,5.75,NaN,NaN,NaN,5.00,5.00,...,Long An,False,False,False,False,False,False,False,False,False
3,24,1,224,8.6,8.50,7.25,9.25,8.5,NaN,NaN,...,Hà Nội,True,True,True,True,False,False,True,False,True
4,24,23,7375,4.4,6.75,NaN,NaN,NaN,5.50,6.75,...,Hòa Bình,False,False,False,False,False,False,False,True,False
5,18,1,36199,4.8,6.25,NaN,NaN,NaN,3.25,4.75,...,Hà Nội,False,False,False,False,False,False,False,False,False
6,18,29,22083,3.6,7.00,NaN,NaN,NaN,7.50,8.00,...,Nghệ An,False,False,False,True,True,True,False,False,False
7,18,57,96,6.0,9.00,NaN,NaN,NaN,6.50,7.25,...,Vĩnh Long,False,False,False,True,False,True,False,True,False
8,22,10,7049,3.6,7.25,NaN,NaN,NaN,7.00,5.00,...,Lạng Sơn,False,False,False,True,True,False,False,True,False
9,21,55,7867,7.6,7.50,8.00,7.00,7.5,NaN,NaN,...,Cần Thơ,True,True,True,True,False,False,True,False,True


Chuyển thành format Boolean DataFrame

In [66]:
subjects = ['toan', 'ly', 'hoa', 'sinh', 'van', 'nn', 'su', 'dia', 'gdcd']
cols = [f'{mon}_gioi' for mon in subjects]
# Drop all columns except the ones we want

df_bool = df_bool.drop(columns=[col for col in df_bool.columns if col not in cols])
df_bool.head(10)

,toan_gioi,ly_gioi,hoa_gioi,van_gioi,su_gioi,dia_gioi,sinh_gioi,gdcd_gioi,nn_gioi
0,True,True,False,False,False,False,False,False,True
1,True,True,True,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,True,True,True,True,False,False,True,False,True
4,False,False,False,False,False,False,False,True,False
5,False,False,False,False,False,False,False,False,False
6,False,False,False,True,True,True,False,False,False
7,False,False,False,True,False,True,False,True,False
8,False,False,False,True,True,False,False,True,False
9,True,True,True,True,False,False,True,False,True


Sử dụng thuật toán Apriori để tìm ra các luật kết hợp giữa các môn học với:
- min_support = 0.1: Các tập phổ biến xuất hiện ít nhất 10% trong dữ liệu
- min_confidence = 0.8: Luật có độ tin cậy từ 80% trở lên



In [70]:
freq = apriori(df_bool, min_support=0.1, use_colnames=True)
freq.head(10)

,support,itemsets
0,0.403713,(toan_gioi)
1,0.149987,(ly_gioi)
2,0.149824,(hoa_gioi)
3,0.405052,(van_gioi)
4,0.139711,(su_gioi)
5,0.259700,(dia_gioi)
6,0.510293,(gdcd_gioi)
7,0.190606,(nn_gioi)
8,0.139783,"(toan_gioi, ly_gioi)"
9,0.137555,"(toan_gioi, hoa_gioi)"


- Sort theo lift để xem các luật có độ hấp dẫn cao nhất
- Các luật được xếp theo thứ tự giảm dần của lift
- Đánh giá chất lượng của luật với các độ đo:
    - Support: Tần suất xuất hiện cùng nhau của các môn học
    - Confidence: Độ tin cậy của luật
    - Lift: Độ hấp dẫn của luật

Ý nghĩa các độ đo:
- Support cao: Tập môn học xuất hiện thường xuyên
- Confidence cao: Mối liên hệ mạnh giữa điều kiện và kết quả
- Lift > 1: Luật có ý nghĩa trong thực tế

In [73]:
rules = association_rules(freq, metric="confidence")
rules = rules.sort_values('lift', ascending=False)[['antecedents','consequents','support','confidence','lift']]
rules.head(10)


,antecedents,consequents,support,confidence,lift
0,(ly_gioi),(toan_gioi),0.139783,0.931967,2.308488
1,(hoa_gioi),(toan_gioi),0.137555,0.918110,2.274165
5,"(van_gioi, nn_gioi)",(toan_gioi),0.105495,0.840437,2.081769
2,(nn_gioi),(toan_gioi),0.153635,0.806031,1.996544
8,"(su_gioi, dia_gioi)",(gdcd_gioi),0.107669,0.989069,1.938238
6,"(toan_gioi, dia_gioi)",(gdcd_gioi),0.100487,0.987553,1.935268
7,"(van_gioi, dia_gioi)",(gdcd_gioi),0.151928,0.981265,1.922946
3,(su_gioi),(gdcd_gioi),0.136974,0.980410,1.921271
4,(dia_gioi),(gdcd_gioi),0.250353,0.964008,1.889128


#### A. Luật dự đoán “Toán giỏi”
**Về support**  
- Khoảng 13–15 % học sinh giỏi Lý/Hóa/NN độc lập nằm trong nhóm giỏi Toán; ~10 % nằm trong nhóm giỏi cả Văn + NN.  
- Những nhóm kết hợp (Văn + NN) tuy support hơi thấp hơn, nhưng vẫn ≥ 10 %, đủ phổ biến để quan tâm.

**Về confidence**  
- Nếu giỏi Lý, xác suất giỏi Toán lên tới ~93 %.  
- Tương tự, giỏi Hóa → giỏi Toán ~91 %; giỏi NN → giỏi Toán ~81 %; và giỏi cả Văn + NN cũng đến ~84 %.

**Về lift**  
- Lift ≈ 2.3–2.0 cho thấy:  
- “Một học sinh giỏi Lý sẽ có khả năng giỏi Toán cao gấp ~2.3 lần so với một học sinh ngẫu nhiên.”  
- Lift > 2 là mối liên hệ rất mạnh, không phải do ngẫu nhiên hay vì Toán giỏi vốn rất phổ biến.

**Nhận xét**  
- Tư duy toán học gắn chặt với tư duy Vật lý và Hóa học. Nếu em nào đã giỏi Lý/Hóa, khả năng rất cao em ấy cũng có thể học được những bài toán khó và đạt điểm cao trong kì thi.
- Kết hợp Văn + ngoại ngữ cũng có khả năng gợi ý năng lực Toán. Có thể những học sinh học giỏi cả văn và ngoại ngữ là những sinh viên học giỏi đều (all-rounder)

---

#### B. Luật dự đoán “GDCD giỏi”
**Support & Confidence**  
- Đơn môn: nếu giỏi Sử, ~13.7 % thí sinh, thì 98 % trong số đó còn giỏi cả GDCD.  
- Hai môn kết hợp (Sử + Địa) hay (Toán + Địa) support ≈ 10 % và confidence rất cao ~98.8–99 %.

**Lift ≈ 1.9**  
- Mặc dù lift thấp hơn một chút so với các luật liên quan đến môn Toán, nhưng vẫn gần gấp đôi so với ngẫu nhiên.

**Nhận xét**  
- Học sinh đa số dễ dàng đạt được điểm cao GDCD nếu học giỏi một tổ hợp môn gồm (Địa + môn X) trong đó X có thể là (Sử, Toán hoặc Văn).

---

## Kết luận & Ứng dụng

- Các luật “giỏi” này giúp chúng ta có thể **tự động phân loại học sinh** tiềm năng cho từng mảng ôn tập nâng cao:  
  - Nhóm giỏi Lý/Hóa/Ngoại ngữ → khuyến khích luyện tập với những bài Toán khó.  
  - Nhóm giỏi Sử/Địa → cần cố gắng luyện tập để điểm GDCD cao.  
- **Lift > 1.9** kết hợp với **confidence > 0.94** cho thấy mối liên hệ không chỉ phổ biến mà còn rất đặc trưng.  
- Kết quả này có thể dùng để:  
  1. **Tự động gợi ý**: khi nhập điểm Sử, Địa, hệ thống tự động đánh dấu em ấy cần chú trọng GDCD. Vì đa số học sinh đều đạt điểm cao GDCD khi điểm Sử/Địa cao. 
  2. **Thiết kế lộ trình học**: ví dụ “Nếu bạn giỏi Sử & Địa, giáo viên có thể cho học sinh thử bộ đề GDCD nâng cao”.  